In [2]:
import json
import stanza

from stanza.server import CoreNLPClient

In [2]:
# run once to install CoreNLP
stanza.install_corenlp()

# 1 - Entity extraction
"Entities are represented as a triple containing: a head word <i>h</i>, a list <i>A</A> of attribute relations, and a list <i>T</i> of trigger relations"

In [3]:
def sample_article():
    file = json.load(open('database_dump_drugs/0.json'))
    return file[0]['article_content']

article = sample_article()
sent = article[:114]  # first sentence of the article

## 1.1 Head word extraction
The paper states that "Head words are extracted from noun phrases."  
Noun phrases are defined as: "A word or group of words containing a noun and functioning in a sentence as subject, object, or prepositional object."

In [ ]:
def head_words(text):
    # finds single head words from noun phrases in a given sentence

    # extract noun phrases
    with CoreNLPClient(properties='corenlp_server-2e15724b8064491b.props') as client:
        pattern = 'NP'
        matches = client.tregex(text=text, pattern=pattern)
    noun_phrases = [sentence[match_id]['spanString'] for sentence in matches['sentences'] for match_id in sentence]

    # extract noun phrases that contain multiple nouns
    doubles = []
    for sentence in matches['sentences']:
        match_ids = range(len(sentence))
        for m in match_ids:
            # for each noun phrase m
            for n in match_ids:
                # check if there is another noun phrase n that has both start >= and end <= m
                if (sentence[str(n)]['characterOffsetBegin'] >= sentence[str(m)]['characterOffsetBegin']) \
                        & (sentence[str(n)]['characterOffsetEnd'] <= sentence[str(m)]['characterOffsetEnd']) \
                        & (m != n) \
                        & (m not in doubles):
                    # if so, then m contains n, so we only want to keep n
                    doubles.append(m)
    for n in doubles:
        del(noun_phrases[n])

    # turn noun phrases into single head words
    head_words = []
    nlp = stanza.Pipeline(lang='en', processors='tokenize,pos,lemma,depparse')
    for phrase in noun_phrases:
        doc = nlp(phrase.lower())
        for sent in doc.sentences:
            for word in sent.words:
                if word.deprel == "root":
                    head_words.append(word.text)
    return head_words

## 1.2 Trigger extraction

In [ ]:
# 1) get verbs

# 2) get 'eventive' nouns from the two wordnet synsets

## 1.3 Drop head words that are not related to at least one trigger

In [ ]:
# some more code

## 1.4 Extract head word attributes

In [ ]:
# some more code